## Part 1: Find Dataset

Dataset should fulfil the following criteria:

1. Clean speech

2. Spontaneous speech

3. Diverse in terms of types of speakers

Use HuggingFace datasets to load the dataset

Look at using the [GigaSpeech](https://huggingface.co/datasets/speechcolab/gigaspeech) dataset first because it has
- High quality labeled audio (some clean some contains noise)
- Diverse in terms of domains and so speaker styles should be diff: audiobooks, podcast, youtube
- Both read and spontaneous speaking styles



In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
#Login because of gated access
#Token is saved to cache and available across all huggingface_hub components
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# Load the dataset
# gigapseech takes about 5mins
from datasets import load_dataset
ds_train = load_dataset("speechcolab/gigaspeech", "xs")

data/xs_n_archives.txt:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

data/test_n_archives.txt:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

xs_chunks_0000.tar.gz:   0%|          | 0.00/972M [00:00<?, ?B/s]

dev_chunks_0000.tar.gz:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

test_chunks_0000.tar.gz:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

...

test_chunks_0001_metadata.csv:   0%|          | 0.00/2.66M [00:00<?, ?B/s]

test_chunks_0002_metadata.csv:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
# Seeing the dstrain dataset
ds_train

DatasetDict({
    train: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 9389
    })
    validation: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 6750
    })
    test: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 25619
    })
})

In [7]:
# Load the first example on the fly
ds_train["train"][0]

{'segment_id': 'YOU0000000315_S0000660',
 'speaker': 'N/A',
 'text': "AS THEY'RE LEAVING <COMMA> CAN KASH PULL ZAHRA ASIDE REALLY QUICKLY <QUESTIONMARK>",
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/77906a935e2abb2d9cf4f707fdb5c12b788afa9bc712c39fef49bf186c19af8f/xs_chunks_0000/YOU0000000315_S0000660.wav',
  'array': array([0.0005188 , 0.00085449, 0.00012207, ..., 0.00125122, 0.00076294,
         0.00036621]),
  'sampling_rate': 16000},
 'begin_time': 2941.889892578125,
 'end_time': 2945.070068359375,
 'audio_id': 'YOU0000000315',
 'title': 'Return to Vasselheim | Critical Role: VOX MACHINA | Episode 43',
 'url': 'https://www.youtube.com/watch?v=zr2n1fLVasU',
 'source': 2,
 'category': 24,
 'original_full_path': 'audio/youtube/P0004/YOU0000000315.opus'}

### Mininimum Viable Example for producing a N-best hypotheses

In [ ]:
"""
Most of the available pretrained models have been pretrained on audio datasets with
a sampling rate of 16kHz. In the gigaspeech dataset, it seems like we do not need to
do upsampling

Test the ASR model on a gigaspeech example

Recall that the pipeline is an object that abstracts complex code from the library

Offers simple API dedicated to tasks

A wrapper
"""

# Load the ASR model
from transformers import pipeline
asr = pipeline(task="automatic-speech-recognition", model="openai/whisper-small")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]